In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [2]:
import torch
import numpy as np
import pandas as np
import pickle
from tqdm import tqdm
import sys

In [3]:
import time

time.sleep(10800)

In [4]:
deck1 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-deck2.pth")

In [5]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([421079, 768]), torch.Size([1095951, 768]))

In [6]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [7]:
N = deck2["feat"].size(0)
distdeck = {"dist": [], "level": []}
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][i, None], p=None, dim=1)
        dist_, indices_ = dist_.topk(4, largest=False)
        tlevels = deck1["tlevel"][indices_]

        distdeck["dist"].append(dist_.cpu())
        distdeck["level"].append(tlevels)

        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [2:04:44<00:00, 146.44it/s]


In [8]:
distdeck["dist"] = torch.stack(distdeck["dist"])
distdeck["level"] = torch.stack(distdeck["level"])

In [9]:
torch.save(
    distdeck, "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-distdeck.pth"
)